### HuggingFace Semantic Similarity Estimation
What are we Doing? 
This is essentially a fact finding experiment. 

We want to estimate the possibility of using the current corpus as part of a model to evaluate potential future articles

<div>
   <img src="../../images/SemanticEstimation.png" height="374"; width="374">
</div>

In [1]:
#!pip install sentence_transformers
import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import time
import datetime
from datetime import datetime as DT
import json
import random
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import nltk
# nltk.download('punkt')
# nltk.download('brown')
# Locals 
from pathlib import Path 
import os, sys 
import glob

from IPython.display import Image
from IPython.core.display import HTML 
from sentence_transformers import SentenceTransformer, util

# nltk.download('punkt')
###
module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path)   
from utils import *
import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT
from os import listdir
import os.path as osp
from os.path import isfile, join

# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import pandas as pd
import numpy as np
from dynamoDButils import *
# regex
import re
p = re.compile('[a-z]+')
p
re.compile('[a-z]+')
## LOCAL 
CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
from APIutils import *

###
def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')


/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Cell imports done @ 2022-12-14 22:06:50


In [2]:
%%time

RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

os.environ['AWS_DEFAULT_REGION']= region
print (f"env : {os.getenv('AWS_DEFAULT_REGION')}")
print('\n\nconnecting ...')
exists = table_exists(RESOURCE, REGION, TABLE)
complete(state= "DB read")

env : us-east-1


connecting ...
table dynamodb.Table(name='Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev') status ACTIVE
Table dynamodb.Table(name='Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev') stats: Creation Time : 2022-09-13 02:11:28.285000+00:00

Cell DB read @ 2022-12-14 22:06:52
CPU times: user 72.2 ms, sys: 10.8 ms, total: 83 ms
Wall time: 105 ms


In [ ]:
# schema = get_schema (RESOURCE,TABLE)
# print (f"Schema = {schema}")

In [30]:
resp = extendedScanArticlesNoKey(RESOURCE,TABLE,columns="title,summary")
print (f"Resp length: {len(resp[0])}", end="\n\n")
print (f"Resp[6000] : {resp[6000]}")

page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 4
extendedScanArticlesNoKey: Length: 6246 
Resp length: 2

Resp[6000] : {'summary': {'S': 'TCI\'s stake represents 0.27% of outstanding Alphabet shares, according to Factset data, a position that the hedge fund has steadily accumulated since 2017.TCI noted that headcount has "increased at an annual rate of 20% since 2017," the year that TCI first disclosed their Alphabet position.TCI argued for an increase in share buybacks and the establishment of an EBIT margin target for Google Services.Significantly, TCI argued that Google\'s "Other Bets" category – their Moonshot division – demanded immediate attention, singling out self-driving vertical Waymo as a unit that failed to justify "its excessive investment."Alphabet shares are down more than 30% year-to-date.'}, 'title': {'S': 'Alphabet must cut headcount and trim costs, activist 

In [4]:
resp = extendedScanArticlesWithKey(RESOURCE,TABLE, columns="title,summary", keyword="url")
# print (f"Resp[6126] : {resp[6126]}")

page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 4
extendedScanArticles: Length: 6246 


#### Notes:
The format of each returned item resembles:

    'summary': {'S': 
    "Stock futures are trading up slightly Monday night as investors tried to shake the previous session's selloff.S&P 500 
    futures were fractionally higher while Nasdaq-100 futures fell marginally.Monday's close marked a negative start to the 
    trading week.Market observers are still largely expecting a 50 basis point increase to interest rates at the Fed's 
    December meeting.Investors will look ahead to data Tuesday morning on international trade for insight into the strength 
    of the U.S. and global economy."}, 
    'url': {'S': 'https://www.cnbc.com/2022/12/05/stock-market-futures-open-to-close-news.html'}, 
    'title': {'S': 'Stock futures are up slightly as fears mount over upcoming interest rate hikes'}



In [5]:
for item in resp[:2]:  # print  first 10 articles 
    print (f"URL: {item['url']['S']}\nTitle: {item['title']['S']}:\nSummary: {item['summary']['S']}", end="\n\n")
# TODO:
# clean up the return s/t it does not have stuff like urls or (money, per centage)

URL: https://www.foxbusiness.com/technology/cisco-partners-microsoft-add-teams-meeting-devices
Title: Cisco partners with Microsoft to add Teams to its meeting devices:
Summary: Networking firm Cisco Systems Inc. will add Microsoft Corp.'s Teams messaging app to its meeting devices, the two firms said on Wednesday, offering users an alternative to its own Webex video conferencing app.Ticker Security Last Change Change % CSCO CISCO SYSTEMS INC. 39.27 -0.31 -0.78%Ticker Security Last Change Change % MSFT MICROSOFT CORP. 225.75 +0.34 +0.15%Cisco's Jeetu Patel, head of Security & Collaboration, said the company aims to be the hardware platform for a wide range of conferencing software platforms.MELINDA GATES OPENS UP ON ‘UNBELIEVABLY PAINFUL’ DIVORCE"There's going to be times that people want to jump on a Microsoft Teams call, they want to jump on a Zoom call, they want to jump on a Google call."Ilya Bukshteyn, vice president of Microsoft Teams Calling and Devices, told Reuters the Teams R

#####  Write the return items into a JSON  file s/t we can later read back in and use 

In [6]:
# Serializing json
indx = 0
jsndict = {}
for item in resp:
    jsndict[str(indx)]= {
        "url" : item['url']['S'],
        "title" : item['title']['S'],
        "summary": item['summary']['S']
    }
    indx += 1
    
json_object = json.dumps(jsndict, indent=4)
jsonfile = join(JSONPath, "semanticSimArticles.json")


with open(jsonfile, "w") as outfile:
     outfile.write(json_object)

In [7]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'Records…

In [8]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnf='semanticSimArticles.json' 


In [9]:
jsnfile = osp.join (JSONPath,jsnf)
df = pd.read_json(jsnfile)
xposedf = df.transpose() # pivot the matrix

In [10]:
xposedf

,url,title,summary
0,https://www.foxbusiness.com/technology/cisco-p...,Cisco partners with Microsoft to add Teams to ...,Networking firm Cisco Systems Inc. will add Mi...
1,https://www.cnbc.com/2022/12/06/tim-cook-says-...,Tim Cook says Apple will use chips built in th...,Apple CEO Tim Cook confirmed that Apple will b...
2,https://abcnews.go.com/International/wireStory...,"NATO allies vow to back Moldova, Georgia, and ...",NATO Secretary-General Jens Stoltenberg said a...
3,https://www.digitaltrends.com/mobile/apple-iph...,Your iPhone may be collecting more personal da...,It’s widely believed that iPhones are among th...
4,https://www.cnbc.com/2022/10/05/stocks-making-...,Stocks making the biggest moves in the premark...,Tesla CEO Elon Musk speaks during the official...
...,...,...,...
6241,https://governmentciomedia.com/federal-agencie...,Federal Agencies Implementing New Approaches t...,Modernizing approaches to software security le...
6242,https://governmenttechnologyinsider.com/transi...,Transitioning to Enhanced Networks: Part 4 of ...,"Previously on Future Healthcare Today, our col..."
6243,https://www.cbsnews.com/news/weight-loss-gadge...,A new era of weight-loss gadgets is here — but...,"I felt a special kind of awe, then panic, watc..."
6244,https://www.cnbc.com/2022/10/11/wall-street-is...,Wall Street is bullish on some corners of tech...,Tech stocks have been in a funk for much of th...


In [11]:
summaries = xposedf.summary.to_list()
# summaries[0:10]

In [12]:
%%time
model = SentenceTransformer('all-MiniLM-L6-v2')
complete(state="SentenceTransformer")


Cell SentenceTransformer @ 2022-12-14 22:07:51
CPU times: user 357 ms, sys: 101 ms, total: 459 ms
Wall time: 457 ms


In [13]:
%%time
corpus = summaries
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
complete(state="model.encode")


Cell model.encode @ 2022-12-14 22:08:18
CPU times: user 21.2 s, sys: 7.09 s, total: 28.3 s
Wall time: 23.1 s


In [14]:
jsonfile = "popular.json"
jsonfile2= "popular-with-articles4.json" ## updated 2022.08.02 to brinmg in line with the evovling implementation over on EMR

popsdict = fetchPopularList(jsondata=JSONPath, jsonfile=jsonfile, jsonfile2=jsonfile2, witharticles=True)

pops = popsdict['Popular_Articles']
print(f'pops: {pops},\n\nlen: {len(pops)}')

with_articles: True
pops: ['https://abcnews.go.com/Technology', 'https://ai.googleblog.com/', 'https://arstechnica.com/gadgets/', 'https://blog.executivebiz.com/category/government-technology/', 'https://blogs.cisco.com/innovation', 'https://datainnovation.org/category/blog/', 'https://executivegov.com/category/technology/', 'https://fcw.com', 'https://federalnewsnetwork.com', 'https://federalnewsnetwork.com/category/technology-main/cloud-computing/', 'https://federalnewsnetwork.com/category/technology-main/cybersecurity/', 'https://federalnewsnetwork.com/category/technology-main/it-modernization/', 'https://gcn.com/', 'https://governmentciomedia.com', 'https://governmenttechnologyinsider.com/category/digital-transformation/', 'https://hbr.org/topic/subject/innovation', 'https://innotechtoday.com/', 'https://insideoutside.io/innovation-newsletter/', 'https://interestingengineering.com/innovation', 'https://mashable.com', 'https://newatlas.com/', 'https://news.mit.edu/topic/innovation',

In [15]:
idsWD = widgets.Select(
    options=pops,
    value=pops[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

idsWD.observe(on_change)
display(idsWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('https://abcnews.go.com/…

In [16]:
news = idsWD.value
news

'https://executivegov.com/category/technology/'

In [17]:
%%time
newOnly = False
_paper = newspaper.build(news
        , memoize_articles=newOnly) # TRUE will only crawl deltas
articles = []
categories = []
if _paper.articles:
    print (f"_paper.articles available: {len(_paper.articles)}")
    randomselection = random.randint(0, len(_paper.articles)-1)
    print(f"random selection : {randomselection}")
    for article in _paper.articles:
#         print(f"_paper article url == {article.url}")
        articles.append(article)
else:
    print(f"NO _paper articles")
    
    
categoryurls = _paper.category_urls()
if categoryurls:
    print (f"_paper.category_urls number == {len(categoryurls)}")
    for category in categoryurls:
        print(f"_paper category #  {category}")
        categories.append(category)
else:
    print (f"NO _paper.category_urls")

_paper.articles available: 116
random selection : 40
_paper.category_urls number == 3
_paper category #  https://executivegov.com/category/technology/
_paper category #  https://www.executivegov.com
_paper category #  https://executivegov.com
CPU times: user 218 ms, sys: 20.3 ms, total: 238 ms
Wall time: 1.96 s


In [19]:
def get_titles(articles):
    titles = []
    progress_bar = widgets.IntProgress(min=0, max=len(articles), description="Extracting")
    display(progress_bar)

    for a in articles:
        an_article = Article(url=a.url)
        try:
            an_article.download()
    #         an_article.throw_if_not_downloaded_verbose()
            an_article.parse()
            an_article.nlp()
            title = an_article.title.strip()
            titles.append(title)
        except:
            print (f"Unable to dl article from {a.url}")
            titles.append("NO Article")
        progress_bar.value += 1
    progress_bar.bar_style = "success"
    progress_bar.close()
    return titles

In [20]:
%%time
titles = get_titles(articles)
# titles

IntProgress(value=0, description='Extracting', max=116)

Unable to dl article from http://www.executivegov.com/category/news/
CPU times: user 18.3 s, sys: 136 ms, total: 18.5 s
Wall time: 1min 39s


In [21]:
###
titlesWD = widgets.Select(
    options=titles,
    value=titles[0],
    description='Choose title:',
    disabled=False,
    button_style='success',
    layout=Layout(width='50%', height='256px')# 'success', 'info', 'warning', 'danger' or ''
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

titlesWD.observe(on_change)
display(titlesWD)

Select(description='Choose title:', layout=Layout(height='256px', width='50%'), options=('Executive Gov', 'Law…

In [22]:
print (f"title: {titlesWD.value=}. index = {titlesWD.index}")

title: titlesWD.value='DOD Announces Integration of FAPIIS With System for Award Management'. index = 2


In [23]:
title = ""
summary = ""
keywords = []
top_image = ""
authors = []
url = ""
if _paper.articles:
#     print (f"_paper.articles available: {len(_paper.articles)}")
    print (f"title: {titlesWD.value}. index = {titlesWD.index}")
    a = articles[titlesWD.index]
    print(f"article.url: {a.url} ")
    an_article = Article(url=a.url)
    an_article.download()
    an_article.throw_if_not_downloaded_verbose()
    an_article.parse()
    an_article.nlp()

    title = an_article.title.strip()
    summary = an_article.summary
    keywords = an_article.keywords
    authors = an_article.authors
    top_image = an_article.top_image
    url = a.url

#     print(f"Images    : {an_article.images}" )
#     print(f"Movies    : {an_article.movies}" )
#     print(f"Text      : {an_article.text}" )
else:
    print(f"NO _paper articles")

title: DOD Announces Integration of FAPIIS With System for Award Management. index = 2
article.url: https://executivegov.com/2022/12/dod-announces-integration-of-fapiis-with-system-for-award-management/ 


In [24]:
# Chose the title and url selected above site and incorporate that to test queries and see whether the article scores in scope 
site = url
# TODO 
# (1).get all articles for the site
# (2). add articles to use as queries 
# (3). score each article

print(f"article.url: {site} ")
an_article = Article(url=site)
an_article.download()
an_article.throw_if_not_downloaded_verbose()
an_article.parse()
an_article.nlp()

title = an_article.title.strip()
summary = an_article.summary
keywords = an_article.keywords
authors = an_article.authors
top_image = an_article.top_image

queries = []
# print(f"title: {title}")
print(f"summary: {summary}")
# print(f"keywords: {keywords}")
# print(f"top_image: {top_image}")
queries.append(summary)

article.url: https://executivegov.com/2022/12/dod-announces-integration-of-fapiis-with-system-for-award-management/ 
summary: The office of the undersecretary for acquisition and sustainment at the Department of Defense has released a memorandum announcing the integration of the Federal Awardee Performance and Integrity Information System with the System for Award Management as part of the ongoing modernization of the Integrated Award Environment.
The General Services Administration’s Federal Acquisition Service oversees IAE, which unifies and manages the federal awards process for the government acquisition workforce and business partners and seeks to modernize the award experience through the use of innovative platforms.
“Users can download a report from SAM showing the entity’s basic identification information as well as the entity’s R/Q information.
Additionally, R/Q and proceedings data is available via a new Entity Management application program interface (API) at https://open.gs

In [25]:
%%time
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries[:10]:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    top_score = 0
    
    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))
        
        if score >= top_score:
            top_score = score
            
print("\nHighest Score: {:.4f}".format(top_score))





Query: The office of the undersecretary for acquisition and sustainment at the Department of Defense has released a memorandum announcing the integration of the Federal Awardee Performance and Integrity Information System with the System for Award Management as part of the ongoing modernization of the Integrated Award Environment.
The General Services Administration’s Federal Acquisition Service oversees IAE, which unifies and manages the federal awards process for the government acquisition workforce and business partners and seeks to modernize the award experience through the use of innovative platforms.
“Users can download a report from SAM showing the entity’s basic identification information as well as the entity’s R/Q information.
Additionally, R/Q and proceedings data is available via a new Entity Management application program interface (API) at https://open.gsa.gov/api/entity-api/,” the memo reads.
According to the document, government employees will continue to encode non

In [26]:
# here we want to record the score by adding the following columns to a dataframe:
# url, title, summary, 5 X (idx, corpus[idx], _score}
ddf = pd.DataFrame()
# newdf.iloc[[2434]]

#Make a list of tuples with your data and then create a DataFrame with it:

jsndict = {}
jsndict[site] =  {"summary" : summary}
entries = {"scores" : {}}

# print (f"top_results.type:  {type(top_results)}")
top_r = torch.topk(cos_scores, k=top_k)
scores = top_results[0].cpu().data.numpy()
indices = top_results[1].cpu().data.numpy()
# print(f"scores {scores}")
# print(f"indices {indices}", end="\n\n")
elt = 0
for score, idx in zip(scores, indices):
    score = round(score, 4) # move to cpu memory
#     print( f"Index: {type(idx)}:{idx=}: {corpus[idx]=}\nScore:{type(score)}: {score=}", end="\n\n")
    entry = {
        "corpusindex" : str(idx),
        "score" : str(score),
        "corpusURL" : xposedf.iloc[idx].url,
        "corpusSummary" : xposedf.iloc[idx].summary,
    }
    elt += 1
    entries['scores'].update({'HF'+str(elt) : entry})
    
jsndict[site].update(entries)

    

print (f"results: {jsndict}")

complete(state="done")

results: {'https://executivegov.com/2022/12/dod-announces-integration-of-fapiis-with-system-for-award-management/': {'summary': 'The office of the undersecretary for acquisition and sustainment at the Department of Defense has released a memorandum announcing the integration of the Federal Awardee Performance and Integrity Information System with the System for Award Management as part of the ongoing modernization of the Integrated Award Environment.\nThe General Services Administration’s Federal Acquisition Service oversees IAE, which unifies and manages the federal awards process for the government acquisition workforce and business partners and seeks to modernize the award experience through the use of innovative platforms.\n“Users can download a report from SAM showing the entity’s basic identification information as well as the entity’s R/Q information.\nAdditionally, R/Q and proceedings data is available via a new Entity Management application program interface (API) at https://o

#### Results Dictionary
A typical results dictionary should resemble

    {
          "https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/": {
                "summary": "The Economist has created ...",
                "scores": {
                      "HF1": {
                            "corpusindex": "5197",
                            "score": "0.5627",
                            "corpusURL": "https://newslink.reuters.com/lifestyle/sports/nba-roundup-thunder-rally-4th-stun-mavs-ot-2022-10-30/",
                            "corpusSummary": "Musk has been deluged..."
                      },
                      "HF2": {
                            "corpusindex": "2982",
                            "score": "0.5556",
                            "corpusURL": "https://www.aljazeera.com/news/2022/10/28/106",
                            "corpusSummary": "Former US President..."
                      },
                      "HF3": {
                            "corpusindex": "3678",
                            "score": "0.5153",
                            "corpusURL": "https://www.cnbc.com/2022/11/11/how-to-use-mastodon-i-tried-to-app-heres-what-you-need-to-know.html",
                            "corpusSummary": "It has been reported that ..."
                      },
                      "HF4": {
                            "corpusindex": "5997",
                            "score": "0.5132",
                            "corpusURL": "https://www.washingtonpost.com/business/2022/10/17/kanye-west-parler-acquisition/",
                            "corpusSummary": "On Oct. 17, Kanye West, legally known as Ye, ..."
                      },
                      "HF5": {
                            "corpusindex": "1779",
                            "score": "0.4939",
                            "corpusURL": "https://www.washingtonpost.com/technology/2022/09/27/meta-takes-down-china-operation/",
                            "corpusSummary": "In the network Meta took down,..."
                      }
                }
          }
    }
We can pick apart the dictionary with a few selections, as shown below:

    >>> results['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/']
    
    
	"https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/": {
		"summary": "The Economist ...",
		"scores": {
			"HF1": {
				"corpusindex": "5197",
				"score": "0.5627",
				"corpusURL": "https://newslink.reuters.com/lifestyle/sports/nba-roundup-thunder-rally-4th-stun-mavs-ot-2022-10-30/",
				"corpusSummary": "Musk has been deluged ..."
			},
			"HF2": {
				"corpusindex": "2982",
				"score": "0.5556",
				"corpusURL": "https://www.aljazeera.com/news/2022/10/28/106",
				"corpusSummary": "Former ..."
			},
			"HF3": {
				"corpusindex": "3678",
				"score": "0.5153",
				"corpusURL": "https://www.cnbc.com/2022/11/11/how-to-use-mastodon-i-tried-to-app-heres-what-you-need-to-know.html",
				"corpusSummary": "It has been reported ..."
			},
			"HF4": {
				"corpusindex": "5997",
				"score": "0.5132",
				"corpusURL": "https://www.washingtonpost.com/business/2022/10/17/kanye-west-parler-acquisition/",
				"corpusSummary": "On Oct. 17, ..."
			},
			"HF5": {
				"corpusindex": "1779",
				"score": "0.4939",
				"corpusURL": "https://www.washingtonpost.com/technology/2022/09/27/meta-takes-down-china-operation/",
				"corpusSummary": "In the network Meta took down,..."
			}
		}
	  }
    }
    
 To get at a specific score, one could do:
 
     >>>  jsndict['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/']['scores']['HF1']['score']
     '0.5627'


And so on

In [27]:
jsonfile = join(JSONPath, "semanticSimArticlesTesting.json")
# read this in as an array
element = {}
try:
    with open(jsonfile, "r") as infile:
        element = json.load(infile)
except FileNotFoundError as e:
    print (f"Error : {e}. Creating a new list")
    

element.update(jsndict)

print (len(element.keys()))
json_object = json.dumps(element, indent=4)

with open(jsonfile, "w") as outfile:
     outfile.write(json_object)


7


In [28]:
# How to find a query after we have written it
# For example to run a competing BERTopic evaluation against an existing query

if query in jsndict[site]['summary']:
        print (True)

True


In [29]:
# How to get the keys (i.e., the URLs) from the persistent file
with open(jsonfile, "r") as infile:
    element = json.load(infile)
for key in element.keys():
    print (f"key: {key}")
    print (f"\t\tHigh score: {element[key]['scores']['HF1']['score']}")

key: https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/
		High score: 0.5627
key: https://venturebeat.com/ai/stability-ai-looks-to-aws-cloud-to-power-the-next-generation-of-generative-ai/
		High score: 0.9986
key: https://venturebeat.com/ai/openai-ceo-admits-chatgpt-risks-what-now-the-ai-beat/
		High score: 0.5767
key: https://venturebeat.com/ai/generative-ai-may-be-the-magic-potion-small-marketing-agencies-have-been-waiting-for/
		High score: 0.7521
key: https://www.businessinsider.com/moderna-ceo-interview-on-mrnas-future-in-cancer-care-2022-12
		High score: 0.5688
key: https://www.digitaltrends.com/space/astronaut-spots-spacexs-starbase-facility-from-space/
		High score: 0.5394
key: https://executivegov.com/2022/12/dod-announces-integration-of-fapiis-with-system-for-award-management/
		High score: 0.5135


In [ ]:
### Extra credit bonus: How do we do against something that is already in the newsfeed?
##  Extra extra (for Anna):
#      break out these next several lines and implemen them in APIutils.py

site = "https://www.washingtonpost.com/politics/2022/12/06/covid-vaccine-mandate-military-members-might-be-next-one-fall/"
###
def getQueries(site):
    print(f"article.url: {site} ")
    an_article = Article(url=site)
    an_article.download()
    an_article.throw_if_not_downloaded_verbose()
    an_article.parse()
    an_article.nlp()

    title = an_article.title.strip()
    summary = an_article.summary
    keywords = an_article.keywords
    authors = an_article.authors
    top_image = an_article.top_image

    queries = []
    # print(f"title: {title}")
    print(f"summary: {summary}")
    # print(f"keywords: {keywords}")
    # print(f"top_image: {top_image}")
    queries.append(summary)
    return queries


###


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity

def scoreArticle(corpus, query):
    top_k = min(5, len(corpus))
    for query in queries[:10]:
        query_embedding = model.encode(query, convert_to_tensor=True)

        # We use cosine-similarity and torch.topk to find the highest 5 scores
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")

        top_score = 0

        for score, idx in zip(top_results[0], top_results[1]):
            print(corpus[idx], "(Score: {:.4f})".format(score), "\n\n")

            if score >= top_score:
                top_score = score

    print("\nHighest Score: {:.4f}".format(top_score))
    return zip(top_results[0], top_results[1])

In [ ]:
queries = getQueries(site)

zipresults = scoreArticle(corpus, query)

In [ ]:
## demonstrate we can read the JSON file into a Pandas DataFrame